In [213]:

import pandas as pd
import numpy as np
from datetime import datetime


pd.set_option('display.max_columns', None)
pd.set_option('display.width', 150)


In [214]:

# Update paths as needed
redcap = pd.read_excel("/Users/opethompson/Desktop/Work - Atrium:WFBM/redcap export ILN_DATA_2025-10-06_2353 (1).xlsx")
waiting = pd.read_excel("/Users/opethompson/Desktop/Work - Atrium:WFBM/PATIENTS WAITING ON FOLLOW UP ILN.xlsx", sheet_name="FOLLOW-UP >6mm")
cancer = pd.read_excel("/Users/opethompson/Desktop/Work - Atrium:WFBM/ILN CANCER DIAGNOSIS.xlsx", sheet_name="LUNG CANCER DX - CUMULATIVE")

# Quick check of column names
print("REDCap columns:", redcap.columns.tolist()[:10], "...")
print("Waiting sheet columns:", waiting.columns.tolist()[:10], "...")
print("Cancer sheet columns:", cancer.columns.tolist()[:10], "...")

REDCap columns: ['record_id', 'redcap_repeat_instrument', 'redcap_repeat_instance', 'mrn', 'wakeone_mrn', 'location_ct', 'f_name', 'l_name', 'alt_name', 'dob'] ...
Waiting sheet columns: ['ILN ID LOCATION', 'RedCAP ID', 'ENCOMPASS MRN', 'NAME', 'ALT NAME', 'ALT MRN', 'S FINDING', 'SCAD', 'S AORTA', 'High Risk/Low Risk           *NOTE:  High - current/former smoker'] ...
Cancer sheet columns: ['Context', 'Last Name', 'First Name', 'Date of Birth ', 'MRN', 'Sex', 'ILN Detection Date', 'Nodule Size', 'Confirmed Cancer Date', 'Diagnostic Intervention'] ...


/Users/opethompson/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
/Users/opethompson/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [215]:
waiting_map = {
    "ILN ID LOCATION": "location_ct",
    "RedCAP ID": "record_id",
    "ENCOMPASS MRN": "mrn",
    "NAME": "alt_name",
    "S FINDING": "s_finding",          # new variable
    "CT DATE": "date_ct",
    "NODULE SIZE (mm)": "size",
    "Nodule(s)": "single_mult",
    "Type Nodue": "density",
    "Nodule Location (largest)": "location",
    "LUNG SCREEN ELIGIBLE": "ls_eligible",
    "BRONCH DATE": "bronch_date",      # new variable
    "IR BX DATE": "ir_bx_date",        # new variable
    "NEW NODULE - not previously imaged": "new_nodule",
    "Nodule 1st found": "date_first_found",
    "PCP (Yes) or No": "pcp",
    "PULM REF Y/N": "pulm_ref",
    "ALREADY BEING FOLLOWED BY PULMONOLOGY": "already_follow",
    "Discharge?": "dc_iln",
    "DISCHARGE REASON": "dc_reason",
    "DATE OF DEATH": "date_death"
}

waiting_std = waiting.rename(columns=waiting_map)


In [216]:
cancer_map = {
    "MRN": "mrn",
    "Confirmed Cancer Date": "final_dx_date",
    "Diagnostic Intervention": "final_dx_modality",
    "Lung Cancer": "cancer_type",
    "Other Cancer": "cancer_other",
    "Stage": "cancer_stage",
    "PRIMARY: Surg/Onc/Rad Onc": "treatment_1",
    "Notes": "cancer_tnm",
    "Setting of diagnosis": "context_dx"
}

cancer_std = cancer.rename(columns=cancer_map)
cancer_std["benign_malignant"] = 2  # malignant = 2


In [217]:
# Recode categorical values to REDCap numeric codes

# ---------- 1. Location (ILN ID LOCATION → location_ct) ----------
location_ct_map = {
    "winston ed": 1, "winston other": 2,
    "lex ed": 3, "lex other": 4,
    "hp ed": 5, "hp other": 6,
    "wilkes ed": 7, "wilkes other": 8,
    "wilkes imaging": 9,
    "davie all": 10,
    "wfbi": 11, "wfbi k": 12,
    "westchester": 13,
    "premier": 14
}
waiting_std["location_ct"] = waiting_std["location_ct"].astype(str).str.lower().map(location_ct_map)

# ---------- 2. Nodule Location (largest) ----------
nodule_loc_map = {
    "rul": 1, "rml": 2, "rll": 3, "lul": 4, "lll": 5,
    "other": 6, "unknown": 7
}
waiting_std["location"] = waiting_std["location"].astype(str).str.lower().map(nodule_loc_map)

# ---------- 3. New Nodule ----------
new_nodule_map = {"yes": 1, "no": 2, "y": 1, "n": 2}
waiting_std["new_nodule"] = waiting_std["new_nodule"].astype(str).str.lower().map(new_nodule_map)

# ---------- 4. Nodule(s) Single/Multiple ----------
single_mult_map = {"single": 1, "multiple": 2}
waiting_std["single_mult"] = waiting_std["single_mult"].astype(str).str.lower().map(single_mult_map)

# ---------- 5. PCP ----------
pcp_map = {
    "yes": 1, "y": 1, "true": 1,
    "no": 2, "n": 2, "false": 2,
    "unknown": 3, "": 3, "nan": 3
}
waiting_std["pcp"] = waiting_std["pcp"].astype(str).str.lower().map(pcp_map)

# ---------- 6. Lung Screen Eligible ----------
ls_eligible_map = {"yes": 1, "y": 1, "no": 2, "n": 2, "unknown": 3}
waiting_std["ls_eligible"] = waiting_std["ls_eligible"].astype(str).str.lower().map(ls_eligible_map)

# ---------- 7. Pulmonology Referral ----------
pulm_ref_map = {"yes": 1, "y": 1, "no": 2, "n": 2}
waiting_std["pulm_ref"] = waiting_std["pulm_ref"].astype(str).str.lower().map(pulm_ref_map)

# ---------- 8. Already being followed ----------
already_follow_map = {"yes": 1, "y": 1, "no": 2, "n": 2}
waiting_std["already_follow"] = waiting_std["already_follow"].astype(str).str.lower().map(already_follow_map)

# ---------- 9. Discharge ----------
dc_iln_map = {"yes": 1, "y": 1, "no": 2, "n": 2}
waiting_std["dc_iln"] = waiting_std["dc_iln"].astype(str).str.lower().map(dc_iln_map)

# ---------- 10. Discharge Reason ----------
dc_reason_map = {
    "nodule stable 2+ years": 1,
    "nodule resolution": 2,
    "unable to contact via phone/mail": 3,
    "active oncology patient": 4,
    "already followed by a specialist at ahwfb": 5,
    "already followed by outside facility": 6,
    "care transitioned to outside facility": 7,
    "on/transitioned to hospice": 8,
    "patient refuses further imaging/work up": 9,
    "lung screening patient": 10,
    "deceased": 11,
    "benign - confirmed diagnosis": 12,
    "malignant - confirmed diagnosis": 13,
    "not a nodule (false positive)": 14,
    "other": 15
}
waiting_std["dc_reason"] = waiting_std["dc_reason"].astype(str).str.lower().map(dc_reason_map)

# ---------- 11. Density (Type Nodule) ----------
density_map = {"solid": 1, "subsolid": 2, "ggn": 3, "unsure": 4}
waiting_std["density"] = waiting_std["density"].astype(str).str.lower().map(density_map)

# ---------- Diagnostic Intervention ----------
dx_modality_map = {
    "BRONCHOSCOPY/BIOPSY": 1,
    "CT-GUIDED TRANSTHORACIC BIOPSY": 2,
    "PET/CT - TUMOR BOARD": 3,
    "THORACENTESIS": 4,
    "OTHER EXTRA-THORACIC BIOPSY": 5,
    "PERICARDIOCENTESIS": 6,
    "CRANIOTOMY": 7,
    "SURGICAL RESECTION": 8,
    "SERIAL IMAGING; STABLE >2YR OR RESOLVED": 10,
    "OTHER": 9
}
cancer_std["final_dx_modality"] = cancer_std["final_dx_modality"].astype(str).str.lower().map(dx_modality_map)

# ---------- Lung Cancer Type ----------
cancer_type_map = {
    "nsclc-adenocarcinoma": 1, "nsclc-squamous cell": 2, "squamous cell carcinoma": 3,
    "adenocarcinoma in situ": 4, "sclc": 5, "sclc and nsclc": 6, "non small cell": 7,
    "presumed lung cancer": 8, "neuroendocrine carcinoma of lung": 9, "sarcomatoid carcinoma": 10,
    "carcinoid": 11, "poorly differentiated": 12, "nsclc-not otherwise specified": 13,
    "NON-LUNG PRIMARY (SPECIFY)": 14, "unknown": 15
}
cancer_std["cancer_type"] = cancer_std["cancer_type"].astype(str).str.lower().map(cancer_type_map)

# ---------- Stage ----------
stage_map = {"1": 1, "2": 2, "3": 3, "4": 4, "l - sclc": 7, "e - sclc": 8, "pending": 9, "unknown": 10}
cancer_std["cancer_stage"] = cancer_std["cancer_stage"].astype(str).str.lower().map(stage_map)

# ---------- Treatment ----------
treatment_map = {
    "surgical resection": 1, "sbrt": 2, "chemotherapy": 3, "radiation": 4,
    "chemotherapy+radiation": 5, "chemotherapy+immunotherapy": 6,
    "immunotherapy": 7, "hospice care": 8, "pending": 9,
    "med onc referral": 10, "refused treatment": 11,
    "outside system for treatment": 12, "not candidate for treatment": 13,
    "surgical resection-outside atrium": 14, "hormone therapy": 15
}
cancer_std["treatment_1"] = cancer_std["treatment_1"].astype(str).str.lower().map(treatment_map)

# ---------- Context (Setting of Diagnosis) ----------
context_map = {"inpatient": 1, "outpatient": 2}
cancer_std["context_dx"] = cancer_std["context_dx"].astype(str).str.lower().map(context_map)


In [218]:
# --- STEP: Normalize MRNs and record IDs across all dataframes ---

# --- STEP 1: Normalize column names first ---
waiting.columns = waiting.columns.str.strip().str.lower()
cancer.columns = cancer.columns.str.strip().str.lower()

# --- STEP 2: Fix column name differences for MRN ---
# Waiting sheet uses "encompass mrn"
if "encompass mrn" in waiting.columns:
    waiting["mrn"] = waiting["encompass mrn"]

# Cancer sheet uses "mrn" but capitalized and possibly with spaces
if "mrn" not in cancer.columns:
    # Try to find column containing 'mrn'
    possible_mrn = [c for c in cancer.columns if "mrn" in c.lower()]
    if possible_mrn:
        cancer["mrn"] = cancer[possible_mrn[0]]

# --- STEP 3: Normalize MRN format across all datasets ---
def clean_mrn(value):
    """Standardize MRN-like values to string without .0 or leading zeros."""
    if pd.isna(value):
        return np.nan
    v = str(value).strip().lower()
    if v.endswith(".0"):
        v = v[:-2]
    v = v.lstrip("0")
    return v

# Apply cleaning
for df in [redcap, waiting, cancer]:
    if "mrn" in df.columns:
        df["mrn"] = df["mrn"].apply(clean_mrn)
    if "record_id" in df.columns:
        df["record_id"] = df["record_id"].astype(str).str.strip().str.lower()

# --- STEP 4: Verify consistency ---
print("✅ MRN fields cleaned & standardized.")
print(f"REDCap MRNs: {redcap['mrn'].nunique()}, Waiting MRNs: {waiting['mrn'].nunique()}, Cancer MRNs: {cancer['mrn'].nunique()}")

# Optional sanity preview
print("\nSample MRNs from each source:")
print("REDCap:", redcap['mrn'].dropna().astype(str).head().tolist())
print("Waiting:", waiting['mrn'].dropna().astype(str).head().tolist())
print("Cancer:", cancer['mrn'].dropna().astype(str).head().tolist())


✅ MRN fields cleaned & standardized.
REDCap MRNs: 992, Waiting MRNs: 2221, Cancer MRNs: 166

Sample MRNs from each source:
REDCap: ['22419425', '23195863', '22769991', '5108870', '23029669']
Waiting: ['22419425', '23195863', '22769991', '5108870', '23029669']
Cancer: ['3245919', '23352295', '22686058', '7204335', '18472713']


In [219]:

def normalize_key_values(df):
    """Standardize MRN and record_id values for consistent merging."""
    if "mrn" in df.columns:
        df["mrn"] = (
            df["mrn"]
            .astype(str)
            .str.strip()
            .str.replace(r"\.0$", "", regex=True)
            .str.lstrip("0")
            .str.lower()
        )
    if "record_id" in df.columns:
        df["record_id"] = (
            df["record_id"]
            .astype(str)
            .str.strip()
            .str.replace(r"\.0$", "", regex=True)
            .str.lower()
        )
    return df

# Apply to final DataFrames used in merge
redcap = normalize_key_values(redcap)
waiting_std = normalize_key_values(waiting_std)
cancer_std = normalize_key_values(cancer_std)

In [220]:
def normalize_redcap_names(df):
    if "alt_name" in df.columns:
        def split_alt_name(name):
            if pd.isna(name): return pd.Series([np.nan, np.nan, np.nan])
            name = name.strip()
            if "," in name:
                last, rest = [x.strip() for x in name.split(",", 1)]
                parts = rest.split()
                first = parts[0] if parts else np.nan
                middle = parts[1] if len(parts) > 1 else np.nan
                return pd.Series([first, last, middle])
            else:
                parts = name.split()
                return pd.Series([parts[0] if parts else np.nan, parts[-1] if len(parts)>1 else np.nan, np.nan])
        df[["f_name","l_name","middle_name"]] = df["alt_name"].apply(split_alt_name)
    return df

def normalize_waiting_names(df):
    if "alt_name" not in df.columns:
        return df

    def split_name(name):
        if pd.isna(name):
            return pd.Series([np.nan, np.nan])

        name = str(name).strip()
        parts = name.split()

        if len(parts) == 1:
            
            first = parts[0]
            last  = np.nan
        else:
            first = parts[0]
            last  = parts[-1]  

        return pd.Series([first, last])

    df[["f_name", "l_name"]] = df["alt_name"].apply(split_name)
    return df


def normalize_cancer_names(df):
    if "last name" in df.columns and "first name" in df.columns:
        df["l_name"] = df["last name"].astype(str).str.strip()
        df["f_name"] = df["first name"].astype(str).str.strip()
        df["alt_name"] = df["l_name"] + ", " + df["f_name"]
    return df

redcap  = normalize_redcap_names(redcap)
waiting_std = normalize_waiting_names(waiting_std)
cancer_std  = normalize_cancer_names(cancer_std)

print("✅ Names normalized")


✅ Names normalized


In [221]:
# Drop record_id from Excel-derived datasets so REDCap assigns new IDs
waiting.drop(columns=["record_id"], inplace=True, errors="ignore")
cancer.drop(columns=["record_id"], inplace=True, errors="ignore")


In [222]:
def update_redcap(redcap_df, external_df, key_col="mrn"):
    """Fill blank REDCap cells with non-null Exc# Drop record_id from Excel-derived datasets so REDCap assigns new IDs
waiting.drop(columns=["record_id"], inplace=True, errors="ignore")
cancer.drop(columns=["record_id"], inplace=True, errors="ignore")

# Merge Waiting + Cancer
excel_combined = pd.merge(waiting, cancer, on="mrn", how="outer")
print(f"✅ Excel merged: {excel_combined.shape}")el data; add new MRNs as new rows."""
    updated = redcap_df.copy()

    # --- ensure MRNs are strings ---
    updated[key_col] = updated[key_col].astype(str)
    external_df[key_col] = external_df[key_col].astype(str)

    # --- make MRN index unique for external_df ---
    external_df = (
        external_df
        .drop_duplicates(subset=[key_col], keep="last")  # keep last if duplicate MRNs
        .set_index(key_col)
    )

    # --- fill existing patients ---
    for col in [c for c in external_df.columns if c in updated.columns]:
        mask = updated[key_col].isin(external_df.index)
        temp = external_df[col]
        updated.loc[mask, col] = updated.loc[mask, col].fillna(
            updated.loc[mask, key_col].map(temp)
        )

    # --- add new patients ---
    redcap_keys = set(updated[key_col].dropna())
    new_rows = external_df.loc[~external_df.index.isin(redcap_keys)].reset_index()
    updated = pd.concat([updated, new_rows], ignore_index=True)

    return updated



In [223]:
print("Unique MRNs:")
print("  REDCap:", redcap["mrn"].nunique())
print("  Waiting:", waiting_std["mrn"].nunique())
print("  Cancer:", cancer_std["mrn"].nunique())

overlap_wait = len(set(redcap["mrn"]) & set(waiting_std["mrn"]))
overlap_cancer = len(set(redcap["mrn"]) & set(cancer_std["mrn"]))
print(f"\nMRN overlap with Waiting: {overlap_wait}")
print(f"MRN overlap with Cancer: {overlap_cancer}")

print("\nExample non-matching MRNs (Excel not in REDCap):")
print(list((set(waiting_std["mrn"]) | set(cancer_std["mrn"])) - set(redcap["mrn"]))[:10])


Unique MRNs:
  REDCap: 993
  Waiting: 2222
  Cancer: 166

MRN overlap with Waiting: 987
MRN overlap with Cancer: 94

Example non-matching MRNs (Excel not in REDCap):
['23238656', '23368359', '23063588', '5124079', '4910376', '6370349', '9305928', '23436573', '8738301', '7200047']


In [224]:
# --- Merge waiting + cancer datasets ---
excel_combined = pd.merge(waiting_std, cancer_std, on="mrn", how="outer")

print("Merged Excel dataset shape:", excel_combined.shape)
print("Columns in merged Excel data:", excel_combined.columns.tolist())

valid_cols = set(redcap.columns)
excel_combined = excel_combined[excel_combined.columns.intersection(valid_cols)]


# --- Run update ---
final_df = update_redcap(redcap, excel_combined, "mrn")

final_df = final_df[final_df.columns.intersection(valid_cols)]

# -- cleanup category and dates ---


# ---- LOAD REDCAP DICT + BUILD RULES ----

import pandas as pd
import numpy as np

dd = pd.read_csv("/Users/opethompson/Desktop/Work - Atrium:WFBM/IncidentalLungNodule_DataDictionary_2025-11-02.csv")

# Parse choices into dict
def parse_choices(s):
    """Turn '1, Yes | 2, No' into {'1':'Yes','2':'No'}"""
    if pd.isna(s):
        return None
    out = {}
    for item in str(s).split("|"):
        item = item.strip()
        if "," in item:
            code, label = item.split(",", 1)
            out[code.strip()] = label.strip()
    return out

dd["choices_parsed"] = dd["Choices, Calculations, OR Slider Labels"].apply(parse_choices)

# Build valid category map
valid_map = {
    row["Variable / Field Name"]: list(row["choices_parsed"].keys())
    for _, row in dd.iterrows()
    if isinstance(row["choices_parsed"], dict)
}

# ---- DROP REPEATING INSTRUMENT FIELDS ----

repeat_forms = set(dd.loc[dd["Form Name"].str.contains("biopsy", case=False, na=False), "Form Name"])

repeat_vars = dd.loc[
    dd["Form Name"].isin(repeat_forms), 
    "Variable / Field Name"
].dropna().unique().tolist()

final_df = final_df.drop(columns=[c for c in repeat_vars if c in final_df.columns], errors="ignore")

print(f"✅ Dropped repeating vars: {repeat_vars}")

# ================== FINAL CLEANUP BEFORE EXPORT ==================

def strip_float_strings(val):
    """Convert '7.0' → '7' only if integer-like."""
    if pd.isna(val):
        return np.nan
    s = str(val).strip()
    if s.endswith(".0") and s.replace(".0","").isdigit():
        return s.replace(".0","")
    return s


# --- 1) strip .0 everywhere (safe attempt)
for col in final_df.columns:
    final_df[col] = final_df[col].apply(strip_float_strings)


# --- 2) enforce categorical domain using dictionary
for col, allowed in valid_map.items():
    if col in final_df.columns:
        final_df[col] = final_df[col].astype(str)
        final_df.loc[~final_df[col].isin(allowed), col] = ""  


# --- 3) checkbox fields: convert NaN→0, ensure {"0","1"}
checkbox_fields = [c for c in final_df.columns if "___" in c]

for c in checkbox_fields:
    final_df[c] = final_df[c].replace(np.nan, "0")
    final_df[c] = final_df[c].astype(str).str.replace(".0","", regex=False)
    final_df.loc[~final_df[c].isin(["0","1"]), c] = "0"


# --- 4) completion fields
completion_fields = [c for c in final_df.columns if c.endswith("_complete")]

for c in completion_fields:
    final_df[c] = final_df[c].astype(str).str.replace(".0","", regex=False)
    final_df.loc[~final_df[c].isin(["", "0", "1", "2"]), c] = ""


# --- 5) DATE formatting → MM/DD/YYYY
date_cols = [c for c in final_df.columns if "date" in c.lower()]
for c in date_cols:
    final_df[c] = (
        pd.to_datetime(final_df[c], errors="coerce")
        .dt.strftime("%m/%d/%Y")
    )
    final_df[c] = final_df[c].replace("NaT", "")
    
# === DOB FIX ===
if "dob" in final_df.columns:
    final_df["dob"] = (
        pd.to_datetime(final_df["dob"], errors="coerce")
        .dt.strftime("%m/%d/%Y")
    )
    final_df["dob"] = final_df["dob"].replace("NaT", "")



# --- 6) Remove literal "nan"
final_df = final_df.replace("nan", "")
final_df = final_df.replace(np.nan, "")

print("✅ Final cleanup complete")


# === FIX EMPTY RECORD_IDs ===
final_df["record_id"] = final_df["record_id"].replace("", np.nan)

# Convert to numeric if possible
existing_ids = pd.to_numeric(final_df["record_id"], errors="coerce")
max_id = int(existing_ids.max())

needs_id = final_df["record_id"].isna()

# MRN → record_id map for existing assignments
mrn_map = (
    final_df.loc[~needs_id, ["mrn", "record_id"]]
    .dropna()
    .drop_duplicates(subset="mrn")
    .set_index("mrn")["record_id"]
    .to_dict()
)

current = max_id

for idx, row in final_df.loc[needs_id].iterrows():
    mrn = row.get("mrn", "")
    
    # Reuse existing
    if mrn in mrn_map:
        final_df.at[idx, "record_id"] = mrn_map[mrn]
    else:
        # Create new
        current += 1
        final_df.at[idx, "record_id"] = str(current)
        mrn_map[mrn] = str(current)

print("✅ Assigned new record_id up to", current)


# --- Export ---
output_path = "/Users/opethompson/Desktop/Work - Atrium:WFBM/redcap_import_ready.csv"
final_df.to_csv(output_path, index=False)

print(f"✅ Export complete: {output_path}")
print("Final REDCap import file shape:", final_df.shape)


Merged Excel dataset shape: (2981, 87)
Columns in merged Excel data: ['location_ct', 'record_id', 'mrn', 'alt_name', 'ALT NAME', 'ALT MRN', 's_finding', 'SCAD', 'S AORTA', 'High Risk/Low Risk           *NOTE:  High - current/former smoker', 'REMINDER DATE', 'date_ct', 'size', 'single_mult', 'density', 'location', 'ls_eligible', 'ZIP CODE', 'Repeat CT DATE', 'PET DATE', 'PET avid (Y/N)', 'bronch_date', 'ir_bx_date', 'CANCER DX (Y/N)', 'new_nodule', 'date_first_found', 'pcp', 'pulm_ref', 'already_follow', 'PULM REF APPT DATE', 'HEM/ONC consultation', 'LETTER SENT UNABLE TO REACH PT', 'HEALTH EQUITY NEED (non-English, uninsured, homeless, etc.-please list)', 'Previous under 6mm and grew (moved from DC to NN)', 'notes', 'Bronchoscopy biopsy', 'IR biopsy date', 'Cancer Diagnosed Y/N', 'dc_iln', 'dc_reason', 'date_death', 'Other discharge comment', 'f_name', 'l_name', 'Context', 'Last Name', 'First Name', 'Date of Birth ', 'Sex', 'ILN Detection Date', 'Nodule Size', 'final_dx_date', 'final_d